In [9]:
%load_ext autoreload
%autoreload 2
from slack import SlackInteractor
from llm import LLMInteractor

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [59]:
interactor = SlackInteractor()
data = interactor.fetch_new_messages()

Set conversations_oldest to 1722399936.894109
Loaded 34 old messages
Found 1 new messages
Updated complete_conversations.pkl with new messages


In [60]:
threads = interactor.organize_threads(data)

In [61]:
interactor.get_conversation_stats(data)

,max_len,total_length,total_messages,average_message_size
user_name,,,,
Agentflow2,359,359,1,359.0


In [62]:
threads[0]

{'channel': 'general',
 'thread_ts': Timestamp('2024-08-02 21:03:14.287458897'),
 'messages': [{'ts': Timestamp('2024-08-02 21:03:14.287458897'),
   'user': 'Eli.chen',
   'text': "hey what's your name",
   'is_bot': False,
   'minutes_ago': 1418},
  {'ts': Timestamp('2024-08-02 21:03:17.174019098'),
   'user': 'Agentflow2',
   'text': "My name is Claude. It's nice to meet you! Let me know if you have any questions I can assist you with.",
   'is_bot': True,
   'minutes_ago': 1418},
  {'ts': Timestamp('2024-08-02 21:03:30.199419022'),
   'user': 'Eli.chen',
   'text': 'write a joke about your situation',
   'is_bot': False,
   'minutes_ago': 1418},
  {'ts': Timestamp('2024-08-02 21:03:50.804688931'),
   'user': 'Agentflow2',
   'text': "Here's a joke about my situation as an AI:\n\nWhy don't I ever get tired of answering questions? \nBecause I've got unlimited artificial energy!\n\nI hope that gave you a little chuckle! Let me know if you'd like to hear any other jokes or if there's an

In [64]:
from pandas import Timestamp

llm_interactor = LLMInteractor()
thread = {'channel': 'general',
  'thread_ts': Timestamp('2024-08-01 23:18:53.991878986'),
  'messages': [{'ts': Timestamp('2024-08-01 23:18:53.991878986'),
    'user': 'Eli.chen',
    'text': 'hello everyone. my name is Eli',
    'is_bot': False,
    'minutes_ago': 153}]}
response = llm_interactor.review_and_remind(thread)
response

'No reminders needed at this time.'

In [58]:
for thread in threads:
    if not thread['messages'][-1]['is_bot']:
        response = llm_interactor.generate_thread_response(thread, guideline_prompt)
        interactor.post_thread_reply(thread, response)

Posted reply to thread 1722632063.551869 in channel general
Posted reply to thread 1722632067.855989 in channel general
